In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP),
    RobotConfig(1, RobotType.panda, None,
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.83050454, -0.03022886, -0.55619097,  0.6479769 ],
        [ 0.17364074,  0.9628188 ,  0.20695142, -0.14296415],
        [ 0.5292552 , -0.2684515 ,  0.80487436,  0.07169667],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## init scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## add environment

In [6]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

Please create a subscriber to the marker


In [7]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]})

In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add movable

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

## init planning scene

In [10]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_action import Gripper2Tool, PlacePlane

In [12]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.02), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_object import BoxAction

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxAction, hexahedral=True)
box2= pscene.create_object(oname="box2", gname="box2", _type=BoxAction, hexahedral=True)

## Register object binders

In [16]:
box1.register_binders(pscene, PlacePlane)
box2.register_binders(pscene, PlacePlane)

## Motion Planning

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

In [18]:
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()

In [19]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

In [20]:
ppline.pscene.gscene

## test planning

In [21]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

obj, handle, binder = "box2", "left_p", "goal_bd"
from_state = pscene.get_state(crob.home_pose)
goal_nodes = pscene.get_goal_nodes(from_state.node, obj, binder)

In [22]:
ppline.search(from_state, goal_nodes, verbose=True, display=True, dt_vis=0.01)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor', 'floor')->('floor', 'grip1') = success
Remaining:2->1 / branching: 0->1 (1.25/600.0 s, steps/err: 35(1241.25409126 ms)/0.555779721139)
node: ('floor', 'grip1')->('floor', 'goal') = fail
node: ('floor', 'grip1')->('floor', 'goal') = fail
node: ('floor', 'grip1')->('floor', 'goal') = success
Remaining:1->0 / branching: 1->2 (2.09/600.0 s, steps/err: 16(561.028957367 ms)/0.00203690294572)
======================= terminated 0: first answer acquired ===============================


In [23]:
# ppline.play_schedule(snode_schedule, period = 0.1)

### multiprocess

In [24]:
ppline.search(from_state, goal_nodes, multiprocess=True, verbose=False)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
======================= terminated 0: first answer acquired ===============================
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 5: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired ===============================
======================= terminated 3: first answer acquired from other agent ===============================
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 9

In [25]:
ppline.play_schedule(snode_schedule, period = 0.03)

In [26]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0,0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [27]:
1

1

   Use a production WSGI server instead.
 * Debug mode: off


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/rnb/.local/lib/python2.7/site-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/rnb/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 848, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/home/rnb/.local/lib/python2.7/site-p